In [1]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import os

In [2]:
os.chdir("../")

In [3]:
model = load_model(os.path.join("artifacts/training","model.h5"))

In [5]:
from sklearn.model_selection import train_test_split


data = np.load('artifacts\data_ingestion\data.npz')
X, y = data['X'], data['y']
X = X.reshape((X.shape[0], 50, 50, 1))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
y_val[5]

array([ 2, 14])

In [13]:
z=model.predict(X_val)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step


In [20]:
z[5]

array([ 2.316767, 13.522571], dtype=float32)

In [16]:
test_loss = model.evaluate(X_val, y_val)
print(f'Test loss (Mean Squared Error): {test_loss}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 4.5658
Test loss (Mean Squared Error): 4.364792346954346


In [21]:
import numpy as np
import os
from PIL import Image

# Configuration
num_images = 10  # Set this to your desired number of images
output_folder = 'testimage'
label_file = os.path.join(output_folder, 'labels.txt')

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Generate images and labels
images = []
labels = []
for i in range(num_images):
    image = np.zeros((50, 50), dtype=np.float32)
    x, y = np.random.randint(0, 50, size=2)
    image[x, y] = 255
    images.append(image)
    labels.append([x, y])

    # Save each image as a file
    image_path = os.path.join(output_folder, f'image_{i}.png')
    pil_image = Image.fromarray(image)
    pil_image = pil_image.convert("L")  # Convert to grayscale
    pil_image.save(image_path)

# Save labels to a text file
with open(label_file, 'w') as f:
    for label in labels:
        f.write(f'{label[0]},{label[1]}\n')

print(f"Images and labels saved to '{output_folder}'")


Images and labels saved to 'testimage'


In [22]:
import numpy as np
import os
from PIL import Image

# Configuration
output_folder = 'testimage'
label_file = os.path.join(output_folder, 'labels.txt')

# Load labels
y_test = []
with open(label_file, 'r') as f:
    for line in f:
        x, y = map(int, line.strip().split(','))
        y_test.append([x, y])

# Convert labels to a numpy array
y_test = np.array(y_test)

# Load images
image_files = sorted([f for f in os.listdir(output_folder) if f.endswith('.png')])
X_test = []
for image_file in image_files:
    image_path = os.path.join(output_folder, image_file)
    image = Image.open(image_path)
    image = np.array(image)
    X_test.append(image)

# Convert images to a numpy array
X_test = np.array(X_test)

print(f"Loaded {len(X_test)} images and {len(y_test)} labels.")


Loaded 10 images and 10 labels.


In [32]:
X_test[0].shape

(50, 50)

In [23]:
y_pred = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 635ms/step


In [24]:
y_pred

array([[ 2.6746733, 21.292412 ],
       [39.47131  , 17.465746 ],
       [36.362133 , 20.786829 ],
       [48.106598 , 34.462624 ],
       [44.307014 , 49.803215 ],
       [47.632896 , 31.369806 ],
       [24.07767  ,  3.4519088],
       [ 4.9698944, 37.461727 ],
       [36.918995 , 19.502974 ],
       [ 1.000895 , 44.091095 ]], dtype=float32)

In [25]:
y_test

array([[ 2, 20],
       [38, 19],
       [36, 22],
       [48, 34],
       [45, 49],
       [44, 29],
       [24,  3],
       [ 5, 38],
       [34, 18],
       [ 1, 44]])

In [35]:
import cv2
def preprocess_image(image_path, target_size=(50, 50)):
    # Read the image
    image = cv2.imread(image_path)
    
    # Resize the image
    image = cv2.resize(image, target_size)
    
    # Convert the image to grayscale if it's not already
    if len(image.shape) > 2 and image.shape[2] > 1:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Ensure the image has the correct shape
    if len(image.shape) == 2:  # If grayscale
        image = image.reshape((1,)+image.shape)
    
    return image
print(preprocess_image('testimage\image_0.png').shape)

(1, 50, 50)


In [36]:
print(model.predict([preprocess_image('testimage\image_0.png')]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
[[ 2.6746807 21.29241  ]]


In [42]:
result = model.predict([preprocess_image('testimage\image_0.png')])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


In [43]:
r = str(int(result[0][0]))+" , "+str(int(result[0][1]))

In [44]:
r

'2 , 21'